In [3]:
import pandas as pd

# Load the cleaned DataFrame
df = pd.read_csv("nhl_player_stats_cleaned.csv")

# Convert the "G", "A", and "TP" columns to numeric
df['G'] = pd.to_numeric(df['G'], errors='coerce')
df['A'] = pd.to_numeric(df['A'], errors='coerce')
df['TP'] = pd.to_numeric(df['TP'], errors='coerce')

# Handle cases where "TP" is 0 by setting "G_%" and "A_%" to 0 for those rows
df['G_%'] = df.apply(lambda row: 0 if row['TP'] == 0 else row['G'] / row['TP'], axis=1)
df['A_%'] = df.apply(lambda row: 0 if row['TP'] == 0 else row['A'] / row['TP'], axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv("nhl_player_stats_with_percentages.csv", index=False)

# Display the updated DataFrame
print(df)




           #               Player                  Team    GP     G     A  \
0        1.0       Joe Malone (C)    Montréal Canadiens    20  44.0   4.0   
1        2.0      Cy Denneny (LW)  Ottawa HC (Senators)    21  36.0  10.0   
2        3.0        Reg Noble (C)   Toronto Hockey Club    20  29.0  10.0   
3        4.0    Newsy Lalonde (C)    Montréal Canadiens    14  23.0   7.0   
4        5.0     Corb Denneny (C)   Toronto Hockey Club    21  20.0  10.0   
...      ...                  ...                   ...   ...   ...   ...   
46229  931.0   Ryan Winterton (F)        Seattle Kraken   9.0   0.0   0.0   
46230  932.0   Rem Pitlick (C/RW)    Chicago Blackhawks   9.0   0.0   0.0   
46231  933.0        Zach Dean (C)       St. Louis Blues   9.0   0.0   0.0   
46232  934.0  Robert Bortuzzo (D)                totals  27.0   0.0   0.0   
46233  935.0   Colin White (C/RW)                totals  28.0   0.0   0.0   

         TP   PPG                                         Player_URL  \
0  

In [12]:
import pandas as pd
import numpy as np

def calculate_benchmark(year_group):
    # Sort players by TP in descending order and select the top 10
    top_players = year_group.sort_values(by='TP', ascending=False).head(10).reset_index(drop=True)
    if len(top_players) < 5:
        return None  # Not enough players to calculate benchmarks

    # Calculate percentage gaps below and above each player
    top_players['gap_below'] = top_players['TP'].pct_change(-1).abs() * 100
    top_players['gap_above'] = top_players['TP'].pct_change().abs() * 100

    # Rule 1: Old League Rule
    if top_players.at[0, 'TP'] <= 50:
        return top_players.at[1, 'TP']  # Use the TP of the second-ranked player

    # Rule 2: 7% Rule (only check 2-3-4, 3-4-5, 4-5-6)
    seven_percent_gaps = []
    for i in range(2, 5):  # Check gaps for positions 2-3-4, 3-4-5, 4-5-6
        above_gap = top_players['gap_above'][i] > 7
        below_gap = top_players['gap_below'][i] > 7
        if above_gap and below_gap:
            seven_percent_gaps.append(i)

    if len(seven_percent_gaps) == 1:
        # If a single 7% gap is found, set the benchmark to the TP value where it occurs
        return np.floor(top_players.at[seven_percent_gaps[0], 'TP'])

    elif len(seven_percent_gaps) > 1:
        # If multiple 7% gaps are found, collect and average TP values within the gaps
        tp_values = []
        for gap_index in seven_percent_gaps:
            tp_values.extend(top_players['TP'][gap_index-1:gap_index+1].tolist())
        return np.floor(sum(tp_values) / len(tp_values))

    # Rule 3: 10% Rule (only check 2-3, 3-4, 4-5)
    for i in range(1, 4):  # Check gaps for 2-3, 3-4, 4-5
        gap = top_players['TP'][i] / top_players['TP'][i+1]  # Calculate the gap correctly
        if gap >= 1.10:
            # Average the TP value at the gap and the next one below it, rounded down
            return np.floor((top_players['TP'][i] + top_players['TP'][i+1]) / 2)

    # Default rule: use the TP of the second-ranked player
    return np.floor(top_players.at[1, 'TP'])


df = pd.read_csv("nhl_player_stats_with_percentages.csv")  # Load your data
df['TP'] = pd.to_numeric(df['TP'], errors='coerce')  # Ensure 'TP' is numeric
benchmarks = df.groupby('Year').apply(calculate_benchmark)  # Apply the function
benchmarks.name = 'Year_BM'

# Merge the benchmarks back to the original dataframe
df_with_benchmarks = df.merge(benchmarks, on='Year', how='left')

# Save the results (if needed)
df_with_benchmarks.to_csv("nhl_player_stats_benchmarks.csv", index=False)

/var/folders/s5/cm5s_7l11_z8zz49rgx05bp00000gn/T/ipykernel_99126/3673026687.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  benchmarks = df.groupby('Year').apply(calculate_benchmark)  # Apply the function


In [14]:
import pandas as pd
import numpy as np

# Load data frame
df = pd.read_csv("nhl_player_stats_benchmarks.csv")

# Ensure relevant columns are numeric
df['TP'] = pd.to_numeric(df['TP'], errors='coerce')
df['Year_BM'] = pd.to_numeric(df['Year_BM'], errors='coerce')
df['G_%'] = pd.to_numeric(df['G_%'], errors='coerce')

# Create the "TPB_%" column
df['TPB_%'] = df['TP'] / df['Year_BM']

# Calculate the "MRB" value for the most recent season (2023-2024)
mrb = df.loc[df['Year'] == '2023-2024', 'Year_BM'].iloc[0]

# Create the "ADJ_PTS" column and round down
df['ADJ_PTS'] = np.floor(df['TPB_%'] * mrb + 0.5)

# Create the "ADJ_G" column and round down
df['ADJ_G'] = np.floor(df['ADJ_PTS'] * df['G_%'] + 0.5)

# Create the "ADJ_A" column and round down
df['ADJ_A'] = np.floor(df['ADJ_PTS'] - df['ADJ_G'] + 0.5)

# Save df (if needed)
df.to_csv("nhl_player_stats_with_adjustments.csv", index=False)

In [19]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("nhl_player_stats_with_adjustments.csv")

# Ensure relevant columns are numeric
df['GP'] = pd.to_numeric(df['GP'], errors='coerce')
df['TP'] = pd.to_numeric(df['TP'], errors='coerce')

# Create the "Max_GP" column
def calculate_max_gp(year_group):
    top_20 = year_group.sort_values('TP', ascending=False).head(20)
    return top_20['GP'].mode()[0]  # Find the most common "GP" value

# Apply the function to each "Year" group and merge the result back
max_gp = df.groupby('Year').apply(calculate_max_gp).reset_index()
max_gp.columns = ['Year', 'Max_GP']
df = df.merge(max_gp, on='Year', how='left')

# Create the "GP_%" column
df['GP_%'] = df['GP'] / df['Max_GP']

# Create the "ADJ_GP" column and round down
df['ADJ_GP'] = np.floor(df['GP_%'] * 82 + 0.5)

# Save (if needed)
df.to_csv("nhl_player_stats_with_final_adjustments.csv", index=False)


/var/folders/s5/cm5s_7l11_z8zz49rgx05bp00000gn/T/ipykernel_99126/3088462694.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("nhl_player_stats_with_adjustments.csv")
/var/folders/s5/cm5s_7l11_z8zz49rgx05bp00000gn/T/ipykernel_99126/3088462694.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_gp = df.groupby('Year').apply(calculate_max_gp).reset_index()


In [22]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("nhl_player_stats_with_final_adjustments.csv")

# Create the "ADJ_PPG" column
df['ADJ_PPG'] = df['ADJ_PTS'] / df['ADJ_GP']

# Save the updated DataFrame to a new CSV file
df.to_csv("ep_nhl_player_stats_with_final_adjustments.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("ep_nhl_player_stats_with_final_adjustments")